In [1]:
import torch, numpy as np
from multitask.framework1.multiscale_fusion import StandardMultiScaleFusion
from multitask.framework1.model import MultiTaskFaceAnalysisModel
from backbones.backbones import get_backbone
from multitask.framework1.subnets import FaceRecognitionEmbeddingSubnet, AgeEstimationSubnet, GenderRecognitionSubnet, EmotionRecognitionSubnet, RaceRecognitionSubnet, AttributeRecognitionSubnet, PoseEstimationSubnet

/home/thoal/anaconda3/envs/pytorch/lib/python3.13/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
backbone = get_backbone('swin_v2_t')

/home/thoal/anaconda3/envs/pytorch/lib/python3.13/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
image = torch.randn(size = (8, 3, 112, 112))

In [8]:
multiscale_features = backbone(image)

In [9]:
for i in range(len(multiscale_features)):
    print(multiscale_features[i].shape)

torch.Size([8, 784, 192])
torch.Size([8, 196, 384])
torch.Size([8, 49, 768])
torch.Size([8, 49, 768])


In [10]:
channels = np.array([192, 384, 768, 768])

In [11]:
channels = channels / channels.sum() * 512

In [12]:
channels

array([ 46.54545455,  93.09090909, 186.18181818, 186.18181818])

In [13]:
47 + 93 + 186 + 186

512